# Isochrone Generation

This notebook aims to try and generate isochrones from querying the train station entrances from Supabase
From there it will attempt to craft functions to be used in as .py files

In [1]:
#setting up supabase client
import os
import pandas as pd
from supabase import create_client, Client

#in the py webapp, keys will be stored in github secrets
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

# Querying station entrances from Supabase

We will focus on using entrance lat/long to generate isochrones. In practice, some stations will be missing entrance data, we will need to handle that in practice.

The following codeblock queries from the table 'entrances' from Supabase. Using the lat long columns, it will then create a new column containing shapely Point geometry objects which can be then used to create Geopandas Geodataframes

In [2]:
import geopandas as gpd
from shapely.geometry import Point

#query to supabase using the previous supabase client that was declared
response = supabase.table('entrances').select("*").execute()
data,_ = response

entrances_df = pd.DataFrame(data[1])
# Create a new column in your DataFrame for the geographic data
entrances_df['geometry'] = [Point(xy) for xy in zip(entrances_df['longitude'], entrances_df['latitude'])]

# Convert the DataFrame to a GeoDataFrame
entrances_gdf = gpd.GeoDataFrame(entrances_df, geometry='geometry')
# Set the coordinate reference system (CRS) to EPSG:4326 (WGS84)
entrances_gdf.crs = "EPSG:4326"

entrances_gdf

,entrance_id,longitude,latitude,entrance_destination,entrance_name,geometry
0,3308608988,101.712717,3.158762,None,None,POINT (101.71272 3.15876)
1,3308608989,101.712507,3.158809,None,None,POINT (101.71251 3.15881)
2,4092013971,101.614130,3.022231,None,None,POINT (101.61413 3.02223)
3,4952299503,101.732720,3.104671,None,B,POINT (101.73272 3.10467)
4,5040821684,101.711729,3.146919,None,D,POINT (101.71173 3.14692)
...,...,...,...,...,...,...
289,12155642294,101.792058,2.962773,None,None,POINT (101.79206 2.96277)
290,12155642296,101.791693,2.961973,None,None,POINT (101.79169 2.96197)
291,5042521364,101.727334,3.123862,None,C,POINT (101.72733 3.12386)
292,12182196430,101.727145,3.123381,None,None,POINT (101.72714 3.12338)


In [4]:
#library to call Open Route Service(ORS)'s client and requests
from openrouteservice import client
import os

import sys

# Get the path of the parent directory (the root of the project)
module_path = os.path.abspath(os.path.join('..'))

# Add the parent directory to sys.path
sys.path.append(module_path)

# Now you can import key.py, this is only done for testing purposes. actual live version should be stored in secret github
from key import ORS_KEY

#Personal api_key stored in a key.py file that is in gitignore. Uncomment the following and provide your own ORS api for your own use
## api_key = "you api key " #Provide your personal API key
ors_key: str = ORS_KEY


ors_client = client.Client(key=ors_key) 

In [47]:
[entrances_gdf.iloc[50]['geometry'].coords[0][0],entrances_gdf.iloc[50]['geometry'].coords[0][1]]

[101.6991072, 3.1590919]

In [63]:
isochrone_params = {
              'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
isochrone_params['locations'] = [[entrances_gdf.iloc[50]['geometry'].coords[0][0],entrances_gdf.iloc[50]['geometry'].coords[0][1]]]

temp_iso = ors_client.isochrones(**isochrone_params)


In [64]:
import json
data = json.loads(json.dumps(temp_iso))
df = pd.json_normalize(data['features'])
df = df.drop(columns=['type','properties.group_index'])
df

,properties.value,properties.center,properties.area,properties.reachfactor,properties.total_pop,geometry.coordinates,geometry.type
0,300.0,"[101.6991072, 3.1590919]",285653.74,0.8183,2842.0,"[[[101.695956, 3.158645], [101.697156, 3.15670...",Polygon
1,600.0,"[101.6991072, 3.1590919]",1375757.45,0.9853,16064.0,"[[[101.692989, 3.161272], [101.694041, 3.15613...",Polygon
2,900.0,"[101.6991072, 3.1590919]",2935073.01,0.9343,42050.0,"[[[101.692929, 3.163399], [101.692796, 3.16219...",Polygon


In [65]:
df['geometry.coordinates']=df['geometry.coordinates'].apply(lambda row:row[0])

# Remove 'properties.' and 'geometry.' from the column names
df.columns = df.columns.str.replace('properties.', '', regex=False)
df.columns = df.columns.str.replace('geometry.', '', regex=False)
df

,value,center,area,reachfactor,total_pop,coordinates,type
0,300.0,"[101.6991072, 3.1590919]",285653.74,0.8183,2842.0,"[[101.695956, 3.158645], [101.697156, 3.156708...",Polygon
1,600.0,"[101.6991072, 3.1590919]",1375757.45,0.9853,16064.0,"[[101.692989, 3.161272], [101.694041, 3.156137...",Polygon
2,900.0,"[101.6991072, 3.1590919]",2935073.01,0.9343,42050.0,"[[101.692929, 3.163399], [101.692796, 3.16219]...",Polygon


In [66]:
df['entrance_id']=entrances_gdf.iloc[50]['entrance_id']
df['isochrone_id'] = df['entrance_id'].astype(str) + df['value'].astype(int).astype(str)
df

,value,center,area,reachfactor,total_pop,coordinates,type,entrance_id,isochrone_id
0,300.0,"[101.6991072, 3.1590919]",285653.74,0.8183,2842.0,"[[101.695956, 3.158645], [101.697156, 3.156708...",Polygon,7259253706,7259253706300
1,600.0,"[101.6991072, 3.1590919]",1375757.45,0.9853,16064.0,"[[101.692989, 3.161272], [101.694041, 3.156137...",Polygon,7259253706,7259253706600
2,900.0,"[101.6991072, 3.1590919]",2935073.01,0.9343,42050.0,"[[101.692929, 3.163399], [101.692796, 3.16219]...",Polygon,7259253706,7259253706900


In [35]:

df['entrance_id']=entrances_gdf.iloc[0]['entrance_id']
df['isochrone_id'] = df['entrance_id'].astype(str) + df['value'].astype(int).astype(str)
df

,value,center,area,reachfactor,total_pop,coordinates,type,entrance_id,isochrone_id
0,300.0,"[101.6996604, 3.0771713]",175987.32,0.5042,1308.0,"[[101.696648, 3.077026], [101.701746, 3.074257...",Polygon,3308608988,3308608988300
1,600.0,"[101.6996604, 3.0771713]",692574.10,0.4960,6918.0,"[[101.693494, 3.077741], [101.694008, 3.076366...",Polygon,3308608988,3308608988600
2,900.0,"[101.6996604, 3.0771713]",1410623.75,0.4490,17252.0,"[[101.692752, 3.08284], [101.691569, 3.077469]...",Polygon,3308608988,3308608988900


In [67]:
from shapely.geometry import Polygon

# Convert list of coordinates to Polygon objects
df['geometry'] = df['coordinates'].apply(Polygon)
df.drop(columns=['coordinates'],inplace=True)
# Convert to GeoDataFrame
entrances_iso_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")



In [68]:
entrances_iso_gdf

,value,center,area,reachfactor,total_pop,type,entrance_id,isochrone_id,geometry
0,300.0,"[101.6991072, 3.1590919]",285653.74,0.8183,2842.0,Polygon,7259253706,7259253706300,"POLYGON ((101.69596 3.15864, 101.69716 3.15671..."
1,600.0,"[101.6991072, 3.1590919]",1375757.45,0.9853,16064.0,Polygon,7259253706,7259253706600,"POLYGON ((101.69299 3.16127, 101.69404 3.15614..."
2,900.0,"[101.6991072, 3.1590919]",2935073.01,0.9343,42050.0,Polygon,7259253706,7259253706900,"POLYGON ((101.69293 3.16340, 101.69280 3.16219..."


In [69]:
gpf_copy = entrances_iso_gdf.copy()
gpf_copy['geometry'] = gpf_copy['geometry'].apply(lambda geom: geom.wkt)
gpf_copy['area'] = gpf_copy['area'].astype(float)
gpf_copy['reachfactor'] = gpf_copy['reachfactor'].astype(float)
gpf_copy['total_pop'] = gpf_copy['total_pop'].astype(float)



/home/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [70]:
type(gpf_copy['total_pop'][0])

numpy.float64

In [71]:
gpf_copy.to_dict(('records'))

[{'value': 300.0,
  'center': [101.6991072, 3.1590919],
  'area': 285653.74,
  'reachfactor': 0.8183,
  'total_pop': 2842.0,
  'type': 'Polygon',
  'entrance_id': 7259253706,
  'isochrone_id': '7259253706300',
  'geometry': 'POLYGON ((101.695956 3.158645, 101.697156 3.156708, 101.697269 3.156567, 101.697653 3.156408, 101.697895 3.15635, 101.699686 3.156082, 101.702599 3.158124, 101.702744 3.158453, 101.70269 3.158717, 101.702492 3.159303, 101.702226 3.159762, 101.701885 3.159878, 101.701393 3.159998, 101.697245 3.160824, 101.697083 3.160799, 101.696098 3.160538, 101.695822 3.160319, 101.695786 3.160239, 101.695956 3.158645))'},
 {'value': 600.0,
  'center': [101.6991072, 3.1590919],
  'area': 1375757.45,
  'reachfactor': 0.9853,
  'total_pop': 16064.0,
  'type': 'Polygon',
  'entrance_id': 7259253706,
  'isochrone_id': '7259253706600',
  'geometry': 'POLYGON ((101.692989 3.161272, 101.694041 3.156137, 101.694409 3.155563, 101.694618 3.155266, 101.695768 3.154496, 101.69856 3.152631, 10

In [72]:
#upsert will insert the row if it doesn’t exist or update the row if it already exists. 
data, error = supabase.table('entrances_isochrones').upsert(gpf_copy.to_dict(('records'))).execute()



In [11]:
def isochrone_generate(client,parameters,reach_centre):
    import json

    #reach_centre is in the format: [101.7127175, 3.1587619], ie [longitude,latitude]
    parameters['locations'] = reach_centre 
    isochrones_output = client.isochrones(**parameters)

    #convert query output to json
    data_json = json.loads(json.dumps(isochrones_output))

    #json to dataframe
    df = pd.json_normalize(data_json['features'])
    df = df.drop(columns=['type','properties.group_index'])

    #reformate coordinates column
    df['geometry.coordinates']=df['geometry.coordinates'].apply(lambda row:row[0])

    # Remove 'properties.' and 'geometry.' from the column names
    df.columns = df.columns.str.replace('properties.', '', regex=False)
    df.columns = df.columns.str.replace('geometry.', '', regex=False)
    
    return df    